In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Image

In [ ]:
import sys
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from pyjpeg import pyjpeg


image_path = Path('..', 'images', 'xccelerated.JPG')
image = Image.open(image_path)

print('Image size:', image.size)
image

# Convert to Numpy

In [ ]:
im = np.array(image.getdata()).reshape(image.size[::-1] + (3,))

In [ ]:
plt.imshow(im)
plt.axis('off')
plt.tight_layout()
_ = plt.savefig(Path('..', 'images', 'xccelerated_small.JPG'))

# Naive compression

In [ ]:
w, h, _ = im.shape
plt.imshow(im[:int(w/2), :int(h/2)])
plt.axis('off')
plt.tight_layout()
_ = plt.savefig(Path('..', 'images', 'xccelerated_cropped.JPG'))

In [ ]:
w, h, _ = im.shape
plt.imshow(im.clip(0, 127), vmin=0, vmax=127)
plt.axis('off')
plt.tight_layout()
_ = plt.savefig(Path('..', 'images', 'xccelerated_clipped.JPG'))

In [ ]:
im = np.array(image.getdata()).mean(axis=1).reshape(image.size[::-1]).astype(int)

print(f'Pixel values are within {im.min()} and {im.max()}')
print(f'Dtype is', im.dtype)

plt.imshow(im, cmap='gray')
plt.axis('off')
plt.tight_layout()
_ = plt.savefig(Path('..', 'images', 'xccelerated_gray.JPG'))

# DCT

In [ ]:
patch = im[800:808, 600:608]


def matshow(patch, color_func, vmin=None, vmax=None, round_digits=1):
    vmin = vmin or patch.min()
    vmax = vmax or patch.max()
    
    fig, ax = plt.subplots(1, 1)

    ax.matshow(patch, cmap='gray', vmin=vmin, vmax=vmax)
    for v in range(patch.shape[0]):
        for u in range(patch.shape[1]):
            val = round(patch[v, u], round_digits)
            c = color_func(val)
            ax.text(u, v, val, va='center', ha='center', color=c)
            
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])
    plt.tight_layout()
    
    return fig, ax
   
color_func = lambda val: 'white' if val < 110 else 'black'
matshow(patch, color_func)
_ = plt.savefig(Path('..', 'images', 'patch_numbers.JPG'))

In [ ]:
dct_cos = lambda x, u: np.cos(((2 * x + 1) * u * np.pi) / 16)

spatial_frequency = 7
dct_values = dct_cos(np.arange(8), spatial_frequency).reshape(1, 8)

color_func = lambda val: 'white' if val < 0 else 'black'
matshow(dct_values, color_func, vmin=-1, vmax=1)
_ = plt.savefig(Path('..', 'images', f'dct_1d_{spatial_frequency}.JPG'))

In [ ]:
dct_cos = lambda x, u: np.cos(((2 * x + 1) * u * np.pi) / 16)

hor_spatial_frequency = 3
ver_spatial_frequency = 6
hor_dct_values = dct_cos(np.arange(8), hor_spatial_frequency).reshape(1, 8)
ver_dct_values = dct_cos(np.arange(8), ver_spatial_frequency).reshape(8, 1)
dct_values = np.outer(ver_dct_values, hor_dct_values)

matshow(dct_values, color_func, vmin=-1, vmax=1)
_ = plt.savefig(Path('..', 'images', f'dct_2d_{hor_spatial_frequency}_{ver_spatial_frequency}.JPG'))

In [ ]:
fig, axes = plt.subplots(8, 8, figsize=(10, 10))
                              
for v in range(8):
    for u in range(8):                       
        hor_dct_values = dct_cos(np.arange(8), u).reshape(1, 8)
        ver_dct_values = dct_cos(np.arange(8), v).reshape(8, 1)
        dct_values = np.outer(ver_dct_values, hor_dct_values)
        
        axes[v, u].matshow(dct_values, vmin=-1, vmax=1, cmap='gray')
        axes[v, u].get_xaxis().set_ticks([])
        axes[v, u].get_yaxis().set_ticks([])
        
_ = plt.savefig(Path('..', 'images', 'dct_grid.JPG'))

In [ ]:
patch_shifted = patch - 128

color_func = lambda val: 'white' if val < -10 else 'black'
matshow(patch_shifted, color_func)
_ = plt.savefig(Path('..', 'images', 'patch_shifted_numbers.JPG'))

In [ ]:
patch_dct = pyjpeg.dct(patch_shifted)
color_func = lambda val: 'white' if val < -110 else 'black'
matshow(patch_dct.astype(int), color_func)
_ = plt.savefig(Path('..', 'images', 'patch_dct.JPG'))

In [ ]:
fig, axes = plt.subplots(8, 8, figsize=(10, 10))
                              
for v in range(8):
    for u in range(8):                       
        hor_dct_values = dct_cos(np.arange(8), u).reshape(1, 8)
        ver_dct_values = dct_cos(np.arange(8), v).reshape(8, 1)
        dct_values = np.outer(ver_dct_values, hor_dct_values)
        
        c = 'black' if u == 0 and v == 0 else 'white'
        axes[v, u].matshow(dct_values, vmin=-1, vmax=1, cmap='gray')
        axes[v, u].text(
            3.5, 
            3.5, 
            int(patch_dct[v, u]), 
            va='center', 
            ha='center', 
            fontsize=25, 
            color=c,
            weight='bold'
        )
        axes[v, u].get_xaxis().set_ticks([])
        axes[v, u].get_yaxis().set_ticks([])
        
_ = plt.savefig(Path('..', 'images', 'dct_grid_w_numbers.JPG'))

# Check if output matches input

In [ ]:
reconstructed = np.zeros((8, 8))

for v in range(8):
    for u in range(8):                       
        hor_dct_values = pyjpeg._normalization_constant(u) * dct_cos(np.arange(8), u).reshape(1, 8)
        ver_dct_values = pyjpeg._normalization_constant(v) * dct_cos(np.arange(8), v).reshape(8, 1)
        dct_values = np.outer(ver_dct_values, hor_dct_values)
        
        reconstructed += patch_dct[v, u] * dct_values 
         
reconstructed / 4

In [ ]:
patch_shifted

# Decompress image

In [ ]:
quantization = np.array([
    [16, 11, 10, 16, 24, 40, 51, 61], 
    [12, 12, 14, 19, 26, 58, 60, 55], 
    [14, 13, 16, 24, 40, 57, 69, 56], 
    [14, 17, 22, 29, 51, 87, 80, 62], 
    [18, 22, 37, 56, 68, 109, 103, 77], 
    [24, 35, 55, 64, 81, 104, 113, 92], 
    [49, 64, 78, 87, 103, 121, 120, 101], 
    [72, 92, 95, 98, 112, 100, 103, 99], 
])

In [ ]:
quantization_factor = 1
decompressed_im = np.zeros(im.shape)

shifted_im = im - 128

for v in range(0, decompressed_im.shape[0], 8):
    print(v)
    for u in range(0, decompressed_im.shape[1], 8):
        
        patch = shifted_im[v: v + 8, u: u + 8]
        dct = (pyjpeg.dct(patch) / quantization).astype(int) * quantization
        decompressed_im[v: v + 8, u: u + 8] = pyjpeg.idct(dct)
        
decompressed_im += 128

fig, ax = plt.subplots(1, 1)
plt.imshow(decompressed_im, cmap='gray')
ax.get_xaxis().set_ticks([])
ax.get_yaxis().set_ticks([])
plt.tight_layout()
_ = plt.savefig(Path('..', 'images', 'xccelerated_compressed.JPG'))

In [ ]:
fig, ax = plt.subplots(1, 1)
plt.imshow(decompressed_im, cmap='gray')
ax.get_xaxis().set_ticks([])
ax.get_yaxis().set_ticks([])
plt.tight_layout()
_ = plt.savefig(Path('..', 'images', 'xccelerated_compressed.JPG'))